In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tensorflow.keras.models import load_model

In [2]:
def predict_sequence_3(infenc, infdec, source, n_steps):
    # encode
    state_h_1, state_c_1, state_h_2, state_c_2, state_h_3, state_c_3 = infenc.predict(source)  # source_dim = ()
#decoder_input = source[:,-1,:,:,:].reshape((1,1,64,64,1))
    decoder_input = np.repeat(0,128*144).reshape((1,1,128,144,1))
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h_1, c_1, h_2, c_2, h_3, c_3 = infdec.predict([decoder_input, state_h_1, state_c_1, state_h_2, state_c_2, state_h_3, state_c_3])
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state_h_1, state_c_1, state_h_2, state_c_2, state_h_3, state_c_3 = h_1, c_1, h_2, c_2, h_3, c_3
        # update target sequence
        decoder_input = yhat
    return np.array(output)

In [ ]:
Y = np.load("/opt/datos/dataset/gfs/Y.npy")
encoder = load_model('encoder-10e.h5')
decoder = load_model('decoder-10e.h5')
data = Y.reshape(2411,8,137,157,1)
x_test  = data[2160:, 0:4, 0:128, 0:144, :]
y_test  = data[2160:, 4:8, 0:128, 0:144, :]

n = 98
pred3 = predict_sequence_3(encoder, decoder, x_test[np.newaxis,n,:,:,:,:], 4)

In [ ]:
fig, axs = plt.subplots(3, 2)
fig.set_size_inches(14, 16)
axs[0][0] = plt.subplot(321), plt.imshow(y_test[n,0,:,:,:], cmap='Blues'), plt.colorbar(), plt.title('GFS')
axs[1][0] = plt.subplot(323), plt.imshow(y_test[n,1,:,:,:], cmap='Blues'), plt.colorbar(), plt.title('GFS')
axs[2][0] = plt.subplot(325), plt.imshow(y_test[n,2,:,:,:], cmap='Blues'), plt.colorbar(), plt.title('GFS')
axs[0][1] = plt.subplot(322), plt.imshow(pred3[0], cmap='Blues'), plt.colorbar(), plt.title('Prediccion +3hs')
axs[1][1] = plt.subplot(324), plt.imshow(pred3[1], cmap='Blues'), plt.colorbar(), plt.title('Prediccion +6hs')
axs[2][1] = plt.subplot(326), plt.imshow(pred3[2], cmap='Blues'), plt.colorbar(), plt.title('Prediccion +9hs')

plt.show()